# Uniform sampling

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def Call_Price_Heston(S,K,T,r,kappa,theta,nu,rho,V0,alpha=1,L=1000):
    # P= Price of a call with Maturity T and Strike K using the characteristic function of the 
    #    price and the Carr-Madan formula - No FFT used
    # S= Initial price
    # r= risk free rate
    # kappa,theta,nu,rho = parameters Heston. 
    # (kappa: rate vt to theta;  theta: long average variance; 
    # nu: vol of vol;  rho: correlation)
    # V0= initial vol in Heston model
    # alpha = damping factor (alpha >0) typically take alpha=1
    # L = truncation bound for the integral
    import numpy as np
    import scipy.integrate as integrate
    
    i = complex(0,1)
    b=lambda x:(kappa-i*rho*nu*x)
    gamma=lambda x:(np.sqrt(nu**(2)*(x**2+i*x)+b(x)**2))
    a=lambda x:(b(x)/gamma(x))*np.sinh(T*0.5*gamma(x))
    c=lambda x:(gamma(x)*np.cosh(0.5*T*gamma(x))/np.sinh(0.5*T*gamma(x))+b(x))
    d=lambda x:(kappa*theta*T*b(x)/nu**2)

    f=lambda x:(i*(np.log(S)+r*T)*x+d(x))
    g=lambda x:(np.cosh(T* 0.5*gamma(x))+a(x))**(2*kappa*theta/nu**2)
    h=lambda x:(-(x**2+i*x)*V0/c(x))

    phi=lambda x:(np.exp(f(x))*np.exp(h(x))/g(x)) # Characteristic function
    integrand=lambda x:(np.real((phi(x-i*(alpha+1))/((alpha+i*x)*(alpha+1+i*x)))*np.exp(-i*np.log(K)*x)))
    integral = integrate.quad(integrand,0, L)
    P=(np.exp(-r*T-alpha*np.log(K))/np.pi) * integral[0]
    return P

In [3]:
Call_Price_Heston(100,100,1,0,1,0.08,1,-.5,0.08,alpha=1,L=1000) 

8.836959972067445

In [4]:
# choose random values from a given interval
def price_set(S0 = 100, N = 10):
    import random
    import numpy as np
    import pandas as pd
    
    m_ = np.zeros(N) # moneyness
    T_ = np.zeros(N) # time to maturity (in year)
    r_ = np.zeros(N) #risk free rate
    rho_ = np.zeros(N) # correlation
    kappa_ = np.zeros(N) # reversion speed
    gam_ = np.zeros(N) # vol of vol
    nu_ = np.zeros(N) # long average variance
    V0_ = np.zeros(N) # initial variance
    price_ = np.zeros(N)
    
    S = S0 # initial stock price
    
    
    for i in range(N):
        m = random.uniform(0.6, 1.4) # moneyness
        K = S/m
        T = random.uniform(0.05, 3) # time to maturity (in year)
        r = random.uniform(0, 0.05) #risk free rate
        kappa = random.uniform(0, 3) # reversion speed
        nu = random.uniform(0.01, 0.5) # long average variance
        gam = random.uniform(0.01, 0.8) # vol of vol
        rho = random.uniform(-0.9, 0) # correlation
        V0 = random.uniform(0.05, 0.5) # initial variance / vol

        price = Call_Price_Heston(S,K,T,r,kappa,nu,gam,rho,V0)
        
        m_[i] = m # moneyness
        T_[i] = T # time to maturity (in year)
        r_[i] = r #risk free rate
        rho_[i] = rho # correlation
        kappa_[i] = kappa  # reversion speed
        gam_[i] = gam # vol of vol
        nu_[i] = nu # long average variance
        V0_[i] = V0 # initial vol
        price_[i] = price
        
    data = [m_, T_, r_, rho_, kappa_, gam_, nu_, V0_, price_]
    df = pd.DataFrame(data, index=['m', 'T', 'r', 'rho', 'kappa', 'gam', 'nu', 'V0', 'price'])
    
    return df.T
       

In [5]:
p = price_set(N = 3)
p

,m,T,r,rho,kappa,gam,nu,V0,price
0,1.083682,0.307056,0.015409,-0.488063,1.121769,0.078022,0.130412,0.163711,13.144815
1,1.305548,2.354216,0.034119,-0.724860,2.552052,0.152811,0.326150,0.129049,NaN
2,1.214501,1.762203,0.000780,-0.578922,2.145089,0.754863,0.105468,0.186177,27.014103


In [6]:
# # Save in .csv
# p.to_csv('price_set.csv')

# import pandas as pd
# pd.read_csv('price_set.csv')

 # LHS 

In [1]:
# Functions
import numpy as np
import random
import pandas as pd

'''
该文件目的是：
1.接收到一组变量范围numpy矩阵以及样本需求个数，shape = (m,2)，输出样本numpy矩阵
执行ParameterArray函数即可
'''

def Partition (number_of_sample,
               limit_array):
    """
    为各变量的变量区间按样本数量进行划分，返回划分后的各变量区间矩阵
    :param number_of_sample: 需要输出的 样本数量
    :param limit_array: 所有变量范围组成的矩阵,为(m, 2)矩阵，m为变量个数，2代表上限和下限
    :return: 返回划分后的个变量区间矩阵（三维矩阵），三维矩阵每层对应于1个变量
    """
    coefficient_lower = np.zeros((number_of_sample, 2))
    coefficient_upper = np.zeros((number_of_sample, 2))
    for i in range(number_of_sample):
        coefficient_lower[i, 0] = 1 - i / number_of_sample
        coefficient_lower[i, 1] = i / number_of_sample
    for i in range(number_of_sample):
        coefficient_upper[i, 0] = 1-(i+1) / number_of_sample
        coefficient_upper[i, 1] = (i+1) / number_of_sample

    partition_lower = coefficient_lower @ limit_array.T  #变量区间下限
    partition_upper = coefficient_upper @ limit_array.T  # 变量区间上限

    partition_range = np.dstack((partition_lower.T, partition_upper.T))  # 得到各变量的区间划分，三维矩阵每层对应于1个变量
    return partition_range #返回区间划分上下限

def Representative(partition_range):
    """
    计算单个随机代表数的函数
    :param partition_range: 一个shape为 (m,N,2) 的三维矩阵，m为变量个数、n为样本个数、2代表区间上下限的两列
    :return: 返回由各变量分区后区间随机代表数组成的矩阵，每列代表一个变量
    """
    number_of_value = partition_range.shape[0]  #获得变量个数
    numbers_of_row = partition_range.shape[1]  # 获得区间/分层个数
    coefficient_random = np.zeros((number_of_value,numbers_of_row, 2))  # 创建随机系数矩阵
    representative_random = np.zeros((numbers_of_row, number_of_value))

    for m in range(number_of_value):
        for i in range(numbers_of_row):
            y = random.random()
            coefficient_random[m,i, 0] = 1 - y
            coefficient_random[m,i, 1] = y

    temp_arr = partition_range * coefficient_random  # 利用*乘实现公式计算（对应位置进行乘积计算），计算结果保存于临时矩阵 temp_arr 中
    for j in range(number_of_value): #计算每个变量各区间内的随机代表数，行数为样本个数n，列数为变量个数m
        temp_random = temp_arr[j, :, 0] + temp_arr[j, :, 1]
        representative_random[:,j] = temp_random
    return representative_random  # 返回代表数向量

def Rearrange(arr_random):
    """
    打乱矩阵各列内的数据
    :param arr_random: 一个N行, m列的矩阵
    :return: 每列打乱后的矩阵
    """
    for i in range(arr_random.shape[1]):
        np.random.shuffle(arr_random[:, i])
    return arr_random



def ParameterArray(limitArray,
                   sampleNumber):
    """
    根据输入的各变量的范围矩阵以及希望得到的样本数量，输出样本参数矩阵
    :param limitArray:变量上下限矩阵，shape为(m,2),m为变量个数
    :param sampleNumber:希望输出的 样本数量
    :return:样本参数矩阵
    """
    arr = Partition(sampleNumber, limitArray)
    parametersMatrix = Rearrange(Representative(arr))
    return  parametersMatrix


'''以下为类创建'''

class DoE(object):
    def __init__(self, name_value, bounds):
        self.name = name_value
        self.bounds = bounds
        self.type = "DoE"
        self.result = None


class DoE_LHS(DoE):
    # 拉丁超立方试验样本生成
    def __init__(self, name_value, bounds, N):
        DoE.__init__(self, name_value, bounds)
        self.type = "LHS"
        self.ParameterArray = ParameterArray(bounds, N)
        self.N = N

    def write_to_csv(self, name):
        """
        将样本数据写入LHS.csv文件，文件保存至运行文件夹内
        """
        sample_data = pd.DataFrame(self.ParameterArray, columns=self.name)
        sample_data.to_csv(name, index=False)

'''以下为使用'''

# arr_limit = np.array([[-100, -100, -100, -1000, -1000, -1000, 0, 32, 8, 100],
#                       [100, 100, 100, 1000, 1000, 100, 10, 2000, 100, 500]]).T
# name_value = ["Fx", "Fy", "Fz", "Mx", "My", "Mz", "Pressure", "R", "nozzle_th", "nozzle_h"]  # 变量名
# q = DoE_LHS(N=100, bounds=arr_limit, name_value=name_value)
# # q.write_to_csv() #样本结果写入csv文件



'以下为使用'

In [11]:
# choose random values from a given interval
def price_set(name, S0 = 100, N = 10):
    arr_limit = np.array([[0.6, 0.05, 0, -0.9, 0.00001, 0.010001, 0.010001, 0.050001],
                          [1.4, 3, 0.05, 0, 3, 0.8, 0.5, 0.5]]).T
    name_value = ['m', 'T', 'r', 'rho', 'kappa', 'gamma', 'nu', 'nu0']  # variable names
    q = DoE_LHS(N=N, bounds=arr_limit, name_value=name_value)
    q.write_to_csv(name) #样本结果写入csv文件
    df = pd.read_csv(name)
    
    df['K'] = S0/df['m']
    df['S'] = S0     
    
    prix = np.zeros(len(df))
    for i in range(len(df)):
        prix[i] = Call_Price_Heston(df['S'][i], df['K'][i], df['T'][i], df['r'][i], df['kappa'][i], 
                                    df['nu'][i],df['gamma'][i], df['rho'][i],df['nu0'][i])
    df['price'] = prix
    
#     df = df.drop(['K', 'S'], axis=1)
    df.dropna(axis=0, how='any', inplace = True)
    df.to_csv(name, index = False)
    
    return df


In [12]:
price_set(name= 'data/param.csv', N = 100).head()

C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in cdouble_scalars
C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp
C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in cdouble_scalars
C:\Users\angie\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficul

,m,T,r,rho,kappa,gamma,nu,nu0,K,S,price
0,1.257914,1.260829,0.046215,-0.307509,0.349958,0.469729,0.183863,0.079676,79.496682,100,28.830581
1,0.834284,2.351793,0.034071,-0.562245,0.151497,0.702222,0.393769,0.081721,119.863229,100,11.736001
11,1.170108,1.718260,0.019274,-0.600835,1.561061,0.438521,0.086567,0.192130,85.462163,100,26.591338
12,0.720118,2.593797,0.018329,-0.553416,0.278247,0.465255,0.109990,0.317225,138.866101,100,16.740601
14,0.909831,0.260397,0.030840,-0.127374,2.050839,0.365112,0.171817,0.244485,109.910492,100,6.163456
15,0.612260,0.611477,0.014961,-0.276206,0.072690,0.310109,0.357146,0.335399,163.329423,100,3.730719
17,0.927662,0.250835,0.013811,-0.831046,1.705972,0.498605,0.153410,0.177375,107.797923,100,4.900558
19,1.280067,0.305467,0.038651,-0.387192,0.708875,0.640925,0.286803,0.196414,78.120891,100,24.800738
20,1.343910,0.218333,0.041936,-0.842385,1.428245,0.290469,0.163926,0.287330,74.409752,100,27.513115
21,1.131024,0.349255,0.043281,-0.659449,2.896892,0.683033,0.020283,0.269836,88.415446,100,17.373866
